### Финальный проект

- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%

- целевая метрика map@5, порого для успешной сдачи map@5 > 20%

- целевая метрика money precision@5 (порог не указан)


Что было сделано:

- настроен prefilter:

       + подобраны способы фильтрации,
       + настроены границы отсечения
       + дополнен utils
        
- дополнен и настроен MainRecommender:

        + подобраны гиперпараметры для ALS
        + выбрано наилучшее взвешивание (bm25) + параметры для него
        + добавлен алгоритм BayesianPersonalisedRanking с подобранными параметрами
        + функция get_top_purchases (возвращает топ своих покупок)
        
- испробованы смешанные варианты генерации кандидатов: 
        
       + некоторые комбинации 
           + в равных долях
           + взвешенные доли - разные варианты взвешивания:
               + с опорой на метрики
               + произвольно

В итоге выбрана смешанная взвешенная модель генерации кандидатов

- генерация кандидатов, признаков и подбор модели 2 уровня
        
        + обработаны имеющиеся и созданы новые признаки
        + выбрана модель lvl_2
        + отобраны признаки
        + подобраны гиперпараметры

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from implicit.bpr import BayesianPersonalizedRanking

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k, ap_at_k, money_precision_at_k
from src.utils import prefilter_items, get_weighted_comb_rec
from src.recommenders import MainRecommender

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')
item_features = pd.read_csv('../raw_data/product.csv')
user_features = pd.read_csv('../raw_data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
# фильтрация
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=2700)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 2701


In [4]:
data_val_lvl_1 = data_val_lvl_1.loc[data_val_lvl_1.user_id.isin(data_train_lvl_1.user_id.unique())]
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


## 1.1. Baselines и варианты моделей 1 уровня

In [5]:
mr = MainRecommender(data_train_lvl_1, weighting='bm25')

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2701 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
k=100

In [7]:
result_lvl_1['top_purchases'] = result_lvl_1['user_id'].apply(lambda x: mr.get_top_purchases(x, N=k))
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: mr.get_als_recommendations(x, N=k))
result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: mr.get_own_recommendations(x, N=k))
result_lvl_1['similar_items'] = result_lvl_1['user_id'].apply(lambda x: mr.get_similar_items_recommendations(x, N=k))
result_lvl_1['similar_users'] = result_lvl_1['user_id'].apply(lambda x: mr.get_similar_users_recommendations(x, N=k))
result_lvl_1['bpr'] = result_lvl_1['user_id'].apply(lambda x: mr.get_bpr_recommendations(x, N=k))

- метрики:
        
        максимизируем recall@100

In [8]:
metrics_100 = pd.DataFrame({'recall@100': [],
                        'precision@100': []
                       })

In [9]:
for col in result_lvl_1.columns[2:]:
    rec = result_lvl_1.apply(lambda row: recall_at_k(row[col], row['actual'], k=k), axis=1).mean()
    prec = result_lvl_1.apply(lambda row: precision_at_k(row[col], row['actual'], k=k), axis=1).mean()
    metrics_100.loc[col] = [rec, prec]

In [10]:
metrics_100

,recall@100,precision@100
top_purchases,0.208979,0.126368
als,0.201346,0.117120
own,0.191839,0.112443
similar_items,0.128693,0.071222
similar_users,0.126747,0.069735
bpr,0.126816,0.073837


## 1.2 Комбинированные алгоритмы

In [11]:
# все алгоритмы в равных долях (по 17 айтемов)
cols = ['top_purchases', 'als', 'own', 'similar_items', 'similar_users', 'bpr']
w = np.array([17]*6)
all_w = w.sum()
weights = (w/all_w)
result_lvl_1['all_weighted'] = result_lvl_1.apply(lambda row: get_weighted_comb_rec(row, cols, weights), axis=1)

In [12]:
# все, взвешенные по recall
cols = ['top_purchases', 'als', 'own', 'similar_items', 'similar_users', 'bpr']
all_w = metrics_100.loc[cols, 'recall@100'].sum()
weights = (metrics_100.loc[cols, 'recall@100']/all_w).values
result_lvl_1['all_weighted_recall'] = result_lvl_1.apply(lambda row: get_weighted_comb_rec(row, cols, weights), axis=1)

In [13]:
# все, взвешенные по precision
cols = ['top_purchases', 'als', 'own', 'similar_items', 'similar_users', 'bpr']
all_w = metrics_100.loc[cols, 'precision@100'].sum()
weights = (metrics_100.loc[cols, 'precision@100']/all_w).values
result_lvl_1['all_weighted_prec'] = result_lvl_1.apply(lambda row: get_weighted_comb_rec(row, cols, weights), axis=1)

In [14]:
# все с произвольными весами - наилучшая точность
cols = ['top_purchases', 'als', 'own', 'similar_items', 'similar_users', 'bpr']
w = np.array([17, 30, 12, 10, 10, 20])
all_w = w.sum()
weights = (w/all_w)
result_lvl_1['all_cust_w'] = result_lvl_1.apply(lambda row: get_weighted_comb_rec(row, cols, weights), axis=1)

In [15]:
# и финальная комбинация - als, own, bpr с произвольными весами
cols = ['als', 'own', 'bpr']
w = np.array([60, 25, 25])
all_w = w.sum()
weights = (w/all_w)
result_lvl_1['als_own_bpr_cust_w'] = result_lvl_1.apply(lambda row: get_weighted_comb_rec(row, cols, weights), axis=1)

In [16]:
for col in result_lvl_1.columns[8:]:
    rec = result_lvl_1.apply(lambda row: recall_at_k(row[col], row['actual'], k=k), axis=1).mean()
    prec = result_lvl_1.apply(lambda row: precision_at_k(row[col], row['actual'], k=k), axis=1).mean()
    metrics_100.loc[col] = [rec, prec]

In [17]:
metrics_100

,recall@100,precision@100
top_purchases,0.208979,0.126368
als,0.201346,0.117120
own,0.191839,0.112443
similar_items,0.128693,0.071222
similar_users,0.126747,0.069735
bpr,0.126816,0.073837
all_weighted,0.132691,0.206251
all_weighted_recall,0.149788,0.184890
all_weighted_prec,0.152243,0.180981
all_cust_w,0.136575,0.201347


- наилучашая точность (неожиданно большая в сравнении с полнотой) - all_weighted (все в равных долях), all_cust_w (произвольные доли для всех алгоритмов) - ненамного отстаёт.
- наилучшая полнота у эвристического top_purchases
- но финальный алгоритм als_own_bpr_cust_w с весами [0.54545455, 0.22727273, 0.22727273] ненамного отстатёт по полноте + не является эвристикой, те для 2 уровня возьмём именно его
- сразу стоит отметить, что top_purchases (baseline) на val_lvl_2 показывает высокие метрики (prec@5=0.38, map@5=0.32). Т.е. модели второго уровня сложно будет её превзойти.

## 2.1 Кандидаты для 2 уровня

In [18]:
k = 100

In [19]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique(), columns=['user_id'])

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['als'] = users_lvl_2['user_id'].apply(lambda x: mr.get_als_recommendations(x, N=k))
users_lvl_2['own'] = users_lvl_2['user_id'].apply(lambda x: mr.get_own_recommendations(x, N=k))
users_lvl_2['bpr'] = users_lvl_2['user_id'].apply(lambda x: mr.get_bpr_recommendations(x, N=k))

In [20]:
cols = ['als', 'own', 'bpr']
w = np.array([60, 25, 25])
all_w = w.sum()
weights = (w/all_w)
weights
users_lvl_2['candidates'] = users_lvl_2.apply(lambda row: get_weighted_comb_rec(row, cols, weights), axis=1)

In [21]:
users_lvl_2.head(2)

,user_id,als,own,bpr,candidates
0,2070,"[1082185, 899624, 826249, 910032, 8065410, 102...","[1085604, 834103, 6534178, 1080414, 1029743, 1...","[1082185, 981760, 1098066, 826249, 995242, 883...","[1082185, 899624, 826249, 910032, 8065410, 102..."
1,2021,"[1044078, 951590, 981760, 1081177, 826249, 844...","[951590, 6534178, 1119454, 1013928, 909894, 95...","[1082185, 981760, 1098066, 826249, 995242, 883...","[1044078, 951590, 981760, 1081177, 826249, 844..."


In [22]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop(['als', 'own', 'bpr', 'candidates'], axis=1).join(s)
users_lvl_2['flag'] = 1

targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

targets_lvl_2.head(3)

,user_id,item_id,target
0,2070,1082185,1.0
1,2070,899624,1.0
2,2070,826249,0.0


In [23]:
# неплохое соотношение классов
targets_lvl_2['target'].mean()

0.176069894861543

## 2.2. Генерация признаков

    - здесь уже только отобранные признаки
    - признаки из user_features для пользователей из targets_lvl_2 - получается чуть менее 2/3 Nan - поэтому не используются
    - к сожалению, тк следующий курс начнётся уже 27.02 - нет времени обернуть в пайплайн

In [24]:
# а в item_features нет пропусков
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')

In [25]:
# среднее число покупок в неделю
avg_purchases_per_w = data_train_lvl_2.groupby(['user_id', 'week_no'])['item_id'].agg(n_items='count').reset_index()
avg_purchases_per_w = avg_purchases_per_w.groupby('user_id')['n_items'].mean().reset_index()
targets_lvl_2 = targets_lvl_2.merge(avg_purchases_per_w, on='user_id', how='left')

In [26]:
# средний чек
avg_check = data_train_lvl_2.groupby(['user_id'])['sales_value'].mean().reset_index().rename(columns={'sales_value': 'avg_ch'})
targets_lvl_2 = targets_lvl_2.merge(avg_check, on='user_id', how='left')

In [27]:
# давность последней транзакции
# но так как размах небольшой (86-91) - можно было бы обойтись и без логарифма
last_tr = data_train_lvl_2.groupby(['user_id'])['week_no'].max().reset_index().rename(columns={'week_no': 'last_tr'})
max_week_no = data_train_lvl_2['week_no'].max()
last_tr['last_tr'] = np.log2(max_week_no + 1 - last_tr['last_tr'])
targets_lvl_2 = targets_lvl_2.merge(last_tr, on='user_id', how='left')

In [28]:
# средня цена в категории
# нет Nan, но есть странаяя категория ' ' с нулевой sales_value
# тк для некоторых товаров есть несколько цен - берём все
avg_s_per_departm = data_train_lvl_2[['item_id', 'sales_value']].merge(item_features[['item_id', 'department']], on='item_id', how='left')
avg_s_per_departm = avg_s_per_departm.groupby('department')['sales_value'].mean().reset_index().rename(columns={'sales_value': 'avg_per_departm'})
# словрь для подстановки
avg_s_per_departm = dict(zip(avg_s_per_departm['department'], avg_s_per_departm['avg_per_departm']))
targets_lvl_2['avg_per_departm'] = targets_lvl_2['department'].replace(avg_s_per_departm)

In [29]:
# транзакций в категории за неделю - без учёта quantity
# нормированное на число уникальных товаров в категории
n_items_per_departm = data_train_lvl_2[['item_id', 'week_no']].merge(item_features[['item_id', 'department']], on='item_id', how='left')
n_items_per_departm = n_items_per_departm.groupby(['department', 'week_no'])['item_id'].count().reset_index().rename(columns={'item_id': 'n_items_per_departm'})
n_items_per_departm = n_items_per_departm.groupby(['department'])['n_items_per_departm'].mean().reset_index()
tmp = data_train_lvl_2[['item_id']].merge(item_features[['item_id', 'department']], on='item_id', how='left')
tmp = tmp.groupby(['department'])['item_id'].nunique().reset_index().rename(columns={'item_id': 'nunique_i'})
n_items_per_departm = n_items_per_departm.merge(tmp, on='department', how='left')
n_items_per_departm['n_items_per_departm'] = n_items_per_departm['n_items_per_departm'] / n_items_per_departm['nunique_i']
# + доп. признак nunique_i - число уникальных айтемов в категории
targets_lvl_2 = targets_lvl_2.merge(n_items_per_departm, on='department', how='left')

In [30]:
# как часто пользователь покупает товары этой категории
# хорошо бы нормировать: число покупок пользователя? число товаров в категории?
freq_purch_per_departm = data_train_lvl_2[['user_id', 'item_id']].merge(item_features[['item_id', 'department']], on='item_id', how='left')
freq_purch_per_departm = freq_purch_per_departm.groupby(['user_id', 'department'])['item_id'].count().reset_index().rename(columns={'item_id': 'freq_purch_per_departm'})
# к числу покупок пользователя
tmp = freq_purch_per_departm.groupby('user_id')['freq_purch_per_departm'].sum().reset_index().rename(columns={'freq_purch_per_departm': 'total_purch'})
freq_purch_per_departm = freq_purch_per_departm.merge(tmp, on='user_id', how='left')
freq_purch_per_departm['freq_purch_per_departm'] = freq_purch_per_departm['freq_purch_per_departm'] / freq_purch_per_departm['total_purch']
# + доп признак - число покупок пользователя
targets_lvl_2 = targets_lvl_2.merge(freq_purch_per_departm, on=['user_id', 'department'], how='left')
targets_lvl_2['freq_purch_per_departm'].fillna(0, inplace=True)
total_purch_med = targets_lvl_2.total_purch.median()
targets_lvl_2['total_purch'].fillna(total_purch_med, inplace=True)

In [31]:
# покупал ли пользователь товары данной категории за последнюю неделю
max_week_no = data_train_lvl_2['week_no'].max()
recently_purch = data_train_lvl_2[['user_id', 'item_id', 'week_no']].merge(item_features[['item_id', 'department']], on='item_id', how='left')
recently_purch = recently_purch.loc[recently_purch['week_no'] == max_week_no].groupby(['user_id', 'department'])['item_id'].first().reset_index().rename(columns={'item_id': 'recently_purch'})
recently_purch['recently_purch'] = 1
targets_lvl_2 = targets_lvl_2.merge(recently_purch, on=['user_id', 'department'], how='left')
targets_lvl_2['recently_purch'] = targets_lvl_2['recently_purch'].fillna(0)

In [32]:
# обратный ранг товара в top_purchases пользователя
top_purch_rev_rank = mr.top_purchases.copy()
top_purch_rev_rank['top_p_rank'] = top_purch_rev_rank.groupby('user_id')['quantity'].rank(method='average', ascending=False)
top_purch_rev_rank['top_p_rank'] = 1 / top_purch_rev_rank['top_p_rank']
targets_lvl_2 = targets_lvl_2.merge(top_purch_rev_rank.drop('quantity', axis=1), on=['user_id', 'item_id'], how='left')
targets_lvl_2['top_p_rank'].fillna(0, inplace=True)

In [33]:
targets_lvl_2.isna().sum().sum()

0

## 2.3. Модель 2 lvl

In [34]:
X_train = targets_lvl_2.drop(['target'], axis=1)
y_train = targets_lvl_2[['target']]

- отобранные признаки (без эмбеддингов: улучшают очень незначительно, но заметно нагружают модель)

In [35]:
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc', 
             'sub_commodity_desc', 'curr_size_of_product']
num_feats = ['n_items', 'avg_ch', 'last_tr', 'avg_per_departm', 'n_items_per_departm', 'nunique_i', 
             'freq_purch_per_departm', 'total_purch', 'recently_purch', 'top_p_rank']

In [36]:
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [37]:
# модель с лучшими гиперпараметрами
lgb = LGBMClassifier(
    objective='binary', 
    max_depth=12,
    n_estimators=75,
    class_weight='balanced',
    min_child_samples=50,
    subsample=0.8,
    colsample_bytree=0.8,
    categorical_column=cat_feats, 
    random_state=42)

In [38]:
# lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats, random_state=42)
lgb.fit(X_train[cat_feats + num_feats], y_train.values.ravel())

LGBMClassifier(categorical_column=['manufacturer', 'department', 'brand',
                                   'commodity_desc', 'sub_commodity_desc',
                                   'curr_size_of_product'],
               class_weight='balanced', colsample_bytree=0.8, max_depth=12,
               min_child_samples=50, n_estimators=75, objective='binary',
               random_state=42, subsample=0.8)

In [39]:
# важность признаков
feats_imp = pd.DataFrame({'feat': lgb.feature_name_,
                          'imortance': lgb.feature_importances_
                         })

feats_imp.sort_values('imortance', ascending=False)

,feat,imortance
4,sub_commodity_desc,614
5,curr_size_of_product,414
0,manufacturer,254
13,total_purch,241
12,freq_purch_per_departm,221
15,top_p_rank,208
3,commodity_desc,164
7,avg_ch,46
6,n_items,43
11,nunique_i,17


- одним из самых сильных признаков оказался top_p_rank (обратный ранг товара в личном топе покупателя): без него метрики не превосходят top_purchases(prec5=0.380304, map5=0.317034)

In [40]:
train_preds_proba = lgb.predict_proba(X_train[cat_feats + num_feats])[:, 1]
train_preds_proba

array([0.90965824, 0.67080934, 0.87409627, ..., 0.03487992, 0.02221697,
       0.01049082])

In [41]:
data_val_lvl_2 = data_val_lvl_2.loc[data_val_lvl_2.user_id.isin(targets_lvl_2.user_id.unique())]

In [42]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']
result_lvl_2.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [43]:
res = pd.concat([targets_lvl_2[['user_id', 'item_id', 'target']].reset_index(drop=True), pd.Series(train_preds_proba)], axis=1).rename(columns={0: 'proba'})
res = res.sort_values(['user_id', 'proba'], ascending=False).groupby('user_id')['item_id'].unique().reset_index().rename(columns={'item_id': 'lgb_lvl_2'})
res.head(3)

,user_id,lgb_lvl_2
0,1,"[1082185, 856942, 995242, 940947, 1075074, 840..."
1,2,"[1106523, 1133018, 8090521, 916122, 1075368, 5..."
2,4,"[883932, 962229, 908283, 1075368, 1029743, 902..."


In [44]:
res['lgb_lvl_2'] = res['lgb_lvl_2'].apply(lambda x: x[:5])
res.head(3)

,user_id,lgb_lvl_2
0,1,"[1082185, 856942, 995242, 940947, 1075074]"
1,2,"[1106523, 1133018, 8090521, 916122, 1075368]"
2,4,"[883932, 962229, 908283, 1075368, 1029743]"


In [45]:
# + рекомендации 2 уровня
result_lvl_2 = result_lvl_2.merge(res, on='user_id', how='left')

In [46]:
# словарь с ценами
item_price_dict = data_train_lvl_1[['item_id', 'quantity', 'sales_value']]
item_price_dict['quantity'].replace({0: 1}, inplace=True)
item_price_dict['price'] = item_price_dict['sales_value'] / item_price_dict['quantity']
item_price_dict = item_price_dict.groupby('item_id')['price'].mean().reset_index()
item_price_dict = dict(zip(item_price_dict['item_id'], item_price_dict['price']))

In [47]:
# + цена
result_lvl_2['prices_recs'] = result_lvl_2['lgb_lvl_2'].apply(lambda recs: [item_price_dict[x] for x in recs])
result_lvl_2.head(2)

,user_id,actual,lgb_lvl_2,prices_recs
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 856942, 995242, 940947, 1075074]","[0.9525739497375202, 2.8046332046332045, 1.289..."
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 1037863, 840361, 1119051, 863447]","[0.9525739497375202, 1.348390943877551, 0.9987..."


In [48]:
# метрики
# recall@5, precision@5, money_precision@5, map@5
metrics_5 = pd.DataFrame({'recall@5': [],
                          'precision@5': [],
                          'money_prec@5': [],
                          'map@5': []
                          })

rec = result_lvl_2.apply(lambda row: recall_at_k(row['lgb_lvl_2'], row['actual'], k=5), axis=1).mean()
prec = result_lvl_2.apply(lambda row: precision_at_k(row['lgb_lvl_2'], row['actual'], k=5), axis=1).mean()
money_prec = result_lvl_2.apply(lambda row: money_precision_at_k(row['lgb_lvl_2'], row['actual'], row['prices_recs'], k=5), axis=1).mean()
map_k = result_lvl_2.apply(lambda row: ap_at_k(row['lgb_lvl_2'], row['actual'], k=5), axis=1).mean()
metrics_5.loc['lgb_lvl_2'] = [rec, prec, money_prec, map_k]

In [49]:
metrics_5 

,recall@5,precision@5,money_prec@5,map@5
lgb_lvl_2,0.054228,0.411906,0.409878,0.34805


In [51]:
# result_lvl_2[['user_id', 'lgb_lvl_2']].to_csv('recommendations.csv', sep=';', index=False, header=False)

Полученные метрики с неплохим запасом превосходят целевые (precision@5 > 25% и map@5 > 20%)